In [4]:
import sys
import os
from pathlib import Path

current_dir = str(Path(os.getcwd()).parent.parent.parent)
print(f"Current dir: {current_dir}")
sys.path.insert(0, current_dir)

Current dir: d:\Programming\Python\comfyui-image-scorer


In [5]:
from shared.paths import filtered_data
from shared.loaders.training_loader import training_loader

retrain = False
if retrain:
    training_loader.remove_training_models()

print("--- Step 1: Loading Data ---")
if not os.path.exists(filtered_data):
    x, y = training_loader.load_training_data()
else:
    x, y = training_loader.load_training_data(load_x=False)


print(f"Loaded Data Shape: {y.shape}")

print("\n--- Step 2: Filtering Unused Features ---")
# Removes features with zero variance or zero importance in a quick probe
x, kept_indices = training_loader.filter_unused_features(x, y, 500)
print(f"Filtered Data Shape: {x.shape}")

print("\n--- Step 3: Generating Interaction Features ---")
# Adds top 500 polynomial interactions (feature_A * feature_B)
x, _ = training_loader.add_interaction_features(x, y, target_k=1000)
print(f"Final Data Shape (with Interactions): {x.shape}")
print("Data Preparation Complete.")

--- Step 1: Loading Data ---
Loaded Data Shape: (15907,)

--- Step 2: Filtering Unused Features ---
Filtered Data Shape: (15907, 3025)

--- Step 3: Generating Interaction Features ---
Final Data Shape (with Interactions): (15907, 4025)
Data Preparation Complete.


In [6]:
import random
from IPython.display import clear_output
from shared.config import config
from  external_modules.step03training.full_data.run import optimize_hyperparameters, evaluate_hyperparameter_combo
from external_modules.step03training.full_data.config_utils import (
    crossover_config,
    generate_random_config,
    #generate_slowest_setup
)
from shared.paths import training_output_dir, models_dir

from shared.loaders.training_loader import training_loader

_,y= training_loader.training_data
x,_=training_loader.interaction_data


# Setup Loop Variables
current_cfg = generate_random_config()
top_cfg = config["training"]["top"]
fastest_cfg = config["training"]["fastest"]
slowest_cfg = config["training"]["slowest"]
#slowest_cfg=generate_slowest_setup()
#config["training"]["slowest"]= slowest_cfg
max_iters = config["training"]["max_iters"]
max_combos = config["training"]["max_combos"]
print(f"Initialized HPO loop with max_iters={max_iters}, max_combos={max_combos}")

temp_model_base = os.path.join(models_dir, "temp_model")

get_base_score=True 
if get_base_score:
    for value in ["top","fastest","slowest"]:
        print(f"getting {value} base score ...")
        current=config["training"][value]
        score, t_time = evaluate_hyperparameter_combo(current , temp_model_base, X=x, y=y)
        new_current = {**current , "best_score": score, "training_time": t_time}
        current.update(new_current)
        print (f"score: {score}")



for i in range(max_iters):
    if i%10==0 and i>0:
        clear_output(wait=True)
    # Refresh references
    top_cfg = config["training"]["top"]
    fastest_cfg = config["training"]["fastest"]
    slowest_cfg = config["training"]["slowest"]

    max_combos = config["training"]["max_combos"]

    # Strategy selection
    rand_val = random.random()
    if rand_val < 0.1:
        base_cfg = generate_random_config()
        strategy = "RANDOM_START"
    elif rand_val < 0.3:
        base_cfg = fastest_cfg
        strategy = "FASTEST"
    elif rand_val < 0.8:
        base_cfg = top_cfg
        strategy = "TOP"
    elif rand_val < 0.9:
         base_cfg = slowest_cfg
         strategy = "SLOWEST"
    else:
        # Crossover
        candidates = [
            c for c in [top_cfg, fastest_cfg, slowest_cfg] if c["best_score"] > -9999
        ]
        if len(candidates) < 2:
            candidates = [c for c in [top_cfg, fastest_cfg, current_cfg] if c]
            if len(candidates) < 2:
                candidates = [generate_random_config(), generate_random_config()]

        parents = random.sample(candidates, 2)
        base_cfg = crossover_config(dict(parents[0]), dict(parents[1]))
        strategy = "CROSSOVER"

    print(f"\nIter {i + 1}/{max_iters} — Strategy: {strategy}")
    print (f"current score; {base_cfg["best_score"]}")
    # optimize_hyperparameters handles updates and saving internally now.
    results = optimize_hyperparameters(
        base_cfg=base_cfg, max_combos=max_combos, X=x, y=y, strategy=strategy
    )

    # Info only loop
    for candidate_cfg, metrics in results:
        r2 = metrics["r2"]
        t_time = metrics["training_time"]

        # Check if this result matches current bests (just for log/info)
        is_top = r2 == top_cfg["best_score"]
        is_fastest = t_time == fastest_cfg["training_time"]

        if is_top:
            print(f"  [Info] This batch produced the current TOP score: {r2:.6f}")
        if is_fastest:
            print(
                f"  [Info] This batch produced the current FASTEST time: {t_time:.4f}s"
            )

    # Refresh local current_cfg for next RANDOM_START or Crossover inheritance
    if results:
        # Simply take the last one as 'current' for random drift
        current_cfg = results[-1][0]

print(f"\nFinished optimization.")
print(f"Top R2: {top_cfg['best_score']:.6f}")
print(
    f"Fastest Time: {fastest_cfg['training_time']:.6f}s (R2: {fastest_cfg['best_score']:.6f})"
)
print(
    f"Slowest (High Score) Time: {slowest_cfg['training_time']:.6f}s (R2: {slowest_cfg['best_score']:.6f})"
)


Iter 11/20 — Strategy: TOP
current score; 0.36083167086926127
last used keys for TOP: ['subsample', 'early_stopping_rounds', 'min_child_samples', 'reg_alpha', 'n_estimators', 'colsample_bytree', 'learning_rate']
Optimizing hyperparameters over grid: {'num_leaves': [492, 403], 'reg_lambda': [5.009064156445117], 'min_split_gain': [0.014335473824042768], 'max_depth': [12], 'subsample': [0.18492329554340467], 'early_stopping_rounds': [72], 'min_child_samples': [314], 'reg_alpha': [1.3760029614604188], 'n_estimators': [1401], 'colsample_bytree': [0.1347357818178288], 'learning_rate': [0.0403707390671591]}
______________________________
Evaluating hyperparameter combo 1/2, with params: {'num_leaves': 492, 'reg_lambda': 5.009064156445117, 'min_split_gain': 0.014335473824042768, 'max_depth': 12, 'subsample': 0.18492329554340467, 'early_stopping_rounds': 72, 'min_child_samples': 314, 'reg_alpha': 1.3760029614604188, 'n_estimators': 1401, 'colsample_bytree': 0.1347357818178288, 'learning_rate':

Training LightGBM: 100%|██████████| 1401/1401 [00:33<00:00, 42.17it/s]


r2=0.3608, time=33.2245s for Evaluated params {'num_leaves': 492, 'reg_lambda': 5.009064156445117, 'min_split_gain': 0.014335473824042768, 'max_depth': 12, 'subsample': 0.18492329554340467, 'early_stopping_rounds': 72, 'min_child_samples': 314, 'reg_alpha': 1.3760029614604188, 'n_estimators': 1401, 'colsample_bytree': 0.1347357818178288, 'learning_rate': 0.0403707390671591}
______________________________
Evaluating hyperparameter combo 2/2, with params: {'num_leaves': 403, 'reg_lambda': 5.009064156445117, 'min_split_gain': 0.014335473824042768, 'max_depth': 12, 'subsample': 0.18492329554340467, 'early_stopping_rounds': 72, 'min_child_samples': 314, 'reg_alpha': 1.3760029614604188, 'n_estimators': 1401, 'colsample_bytree': 0.1347357818178288, 'learning_rate': 0.0403707390671591}


Training LightGBM: 100%|██████████| 1401/1401 [00:32<00:00, 42.57it/s]


r2=0.3608, time=32.9138s for Evaluated params {'num_leaves': 403, 'reg_lambda': 5.009064156445117, 'min_split_gain': 0.014335473824042768, 'max_depth': 12, 'subsample': 0.18492329554340467, 'early_stopping_rounds': 72, 'min_child_samples': 314, 'reg_alpha': 1.3760029614604188, 'n_estimators': 1401, 'colsample_bytree': 0.1347357818178288, 'learning_rate': 0.0403707390671591}
--> Found new TOP model! (old: 0.3608, new: 0.3608)
  [Info] This batch produced the current TOP score: 0.360832

Iter 12/20 — Strategy: TOP
current score; 0.3608324453071855
last used keys for TOP: ['subsample', 'early_stopping_rounds', 'min_child_samples', 'reg_alpha', 'n_estimators', 'colsample_bytree', 'learning_rate', 'num_leaves']
Optimizing hyperparameters over grid: {'max_depth': [13, 10], 'min_split_gain': [0.014335473824042768], 'reg_lambda': [5.009064156445117], 'subsample': [0.18492329554340467], 'early_stopping_rounds': [72], 'min_child_samples': [314], 'reg_alpha': [1.3760029614604188], 'n_estimators':

...

r2=0.3445, time=11.8542s for Evaluated params {'num_leaves': 745, 'early_stopping_rounds': 200, 'learning_rate': 0.11098664421128113, 'min_child_samples': 58, 'reg_lambda': 9.0, 'colsample_bytree': 0.1083359287653048, 'subsample': 0.29627025953957464, 'n_estimators': 2000, 'reg_alpha': 6.842495885918459, 'max_depth': 4, 'min_split_gain': 0.21337314075280728}

Iter 20/20 — Strategy: TOP
current score; 0.36083331323238255
last used keys for TOP: ['early_stopping_rounds']
Optimizing hyperparameters over grid: {'n_estimators': [1541, 1260], 'max_depth': [12], 'reg_lambda': [5.009064156445117], 'subsample': [0.18492329554340467], 'num_leaves': [403], 'min_split_gain': [0.014335473824042768], 'reg_alpha': [1.3760029614604188], 'colsample_bytree': [0.1347357818178288], 'min_child_samples': [314], 'learning_rate': [0.0403707390671591], 'early_stopping_rounds': [79]}
______________________________
Evaluating hyperparameter combo 1/2, with params: {'n_estimators': 1541, 'max_depth': 12, 'reg_lam

Training LightGBM: 100%|██████████| 1541/1541 [00:35<00:00, 43.01it/s]


r2=0.3613, time=35.8300s for Evaluated params {'n_estimators': 1541, 'max_depth': 12, 'reg_lambda': 5.009064156445117, 'subsample': 0.18492329554340467, 'num_leaves': 403, 'min_split_gain': 0.014335473824042768, 'reg_alpha': 1.3760029614604188, 'colsample_bytree': 0.1347357818178288, 'min_child_samples': 314, 'learning_rate': 0.0403707390671591, 'early_stopping_rounds': 79}
--> Found new TOP model! (old: 0.3608, new: 0.3613)
______________________________
Evaluating hyperparameter combo 2/2, with params: {'n_estimators': 1260, 'max_depth': 12, 'reg_lambda': 5.009064156445117, 'subsample': 0.18492329554340467, 'num_leaves': 403, 'min_split_gain': 0.014335473824042768, 'reg_alpha': 1.3760029614604188, 'colsample_bytree': 0.1347357818178288, 'min_child_samples': 314, 'learning_rate': 0.0403707390671591, 'early_stopping_rounds': 79}


Training LightGBM: 100%|██████████| 1260/1260 [00:30<00:00, 41.80it/s]


r2=0.3602, time=30.1464s for Evaluated params {'n_estimators': 1260, 'max_depth': 12, 'reg_lambda': 5.009064156445117, 'subsample': 0.18492329554340467, 'num_leaves': 403, 'min_split_gain': 0.014335473824042768, 'reg_alpha': 1.3760029614604188, 'colsample_bytree': 0.1347357818178288, 'min_child_samples': 314, 'learning_rate': 0.0403707390671591, 'early_stopping_rounds': 79}
  [Info] This batch produced the current TOP score: 0.361284

Finished optimization.
Top R2: 0.361284
Fastest Time: 11.687767s (R2: 0.344496)
Slowest (High Score) Time: 114.289075s (R2: 0.331597)
